Kazhdan-Lustig polynomials of Matroids satisfy a certain formula called the deletion formula. This note will be about obtaining the corresponding deletion formula (bv-klpolydeletion) for the Q-polynomials.

The coeﬃcients of $P_M (t)$ depend only on the lattice of ﬂats $L(M )$, and in fact they are integral linear combinations of the ﬂag Whitney numbers counting chains of ﬂats with speciﬁed ranks.

$$P_M(t) = P_{M\setminus e}(t)-tP_{M/e}(t)+\sum_{F\in S}{\tau(M/{F\cup e})t^{\text{crk}(F)/2}P_{M\mid F}(t)}.$$

Guess 1:
$$Q_M(t) = Q_{M\setminus e}(t)-tQ_{M/e}(t)+\sum_{F\in S}{\tau(M/{F\cup e})t^{\text{crk}(F)/2}Q_{M\mid F}(t)}.$$

In [23]:
M = matroids.Uniform(2, 3)
B = M.orlik_solomon_algebra(QQ)
F = list(B.basis())
F

[OS{}, OS{2}, OS{1}, OS{0}, OS{0, 2}, OS{0, 1}]

In [24]:
def whitney_numbers(M):
    abs_w = [0] * (M.rank()+1)
    for S in M.no_broken_circuits_sets_iterator():
        abs_w[len(S)] += 1
    return [ZZ((-1)**i * val) for i, val in enumerate(abs_w) if val != 0]

In [25]:
def matroid_charpoly(M):
    R = PolynomialRing(QQ, 'x')
    char_poly = R(0)
    Whitney_numbers = M.whitney_numbers()
    for val in Whitney_numbers:
        char_poly += val * x

In [26]:
klpoly_list = []

def get_klpoly(M):

    for item, klpoly in klpoly_list:
        if  M.is_isomorphic(item):
            return klpoly
    
    klpoly = kl_polynomial(M)
    klpoly_list.append((M, klpoly))

    return klpoly

def reduced_char_poly(M):
    return char_poly(M) // (x - 1)

def char_poly(M):
    T = M.tutte_polynomial()
    x = var('x')
    subbed_Tpoly = T.subs(x = 1 - x, y = 0)
    char_poly = (-1) ** M.rank() * subbed_Tpoly
    R = PolynomialRing(QQ, 'x')

    return R(char_poly)

def graphic_char_poly(n):
    T = graphs.CompleteGraph(n).tutte_polynomial()
    x = var('x')
    subbed_Tpoly = T.subs(x = 1 - x, y = 0)
    char_poly = (-1) ** (n - 1) * subbed_Tpoly
    R = PolynomialRing(QQ, 'x')

    return R(char_poly)

def kl_polynomial(M):
    rank = M.rank()
    x = var('x')
    R = PolynomialRing(QQ, 'x')

    if rank == 0: return R(1)
    
    epw_Rpolynomial = R(0)
    
    for i in range(1, rank + 1):
        for flat in M.flats(i):
            restriction = M.delete(M.groundset() - flat).simplify()
            contraction = M.contract(flat).simplify()
            
            restriction_poly = reduced_char_poly(restriction)
            contraction_poly = get_klpoly(contraction)
            
            term_poly = restriction_poly * contraction_poly
            
            epw_Rpolynomial += term_poly

    coeffs_Rpolynomial = epw_Rpolynomial.coefficients(sparse = False)
    coeffs_Rpolynomial = [0] * (rank - epw_Rpolynomial.degree()) + coeffs_Rpolynomial.copy()

    coeffs = kl_poly_inv.coefficients(sparse=False)
    reversed_coeffs = coeffs[::-1]
    kl_poly = R(reversed_coeffs)

    return kl_poly

M = matroids.Uniform(10,10)
result = kl_polynomial(M)
print("KL-polynomial:", result)

NameError: name 'kl_poly_inv' is not defined

In [ ]:
print(klpoly_list)

[(M / {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, where M is U(10, 10): Matroid of rank 10 on 10 elements with circuit-closures
{}, 1), (M / {0, 1, 2, 3, 4, 5, 6, 7, 8}, where M is U(10, 10): Matroid of rank 10 on 10 elements with circuit-closures
{}, 1), (M / {0, 1, 2, 3, 4, 5, 6, 7}, where M is U(10, 10): Matroid of rank 10 on 10 elements with circuit-closures
{}, x + 1), (M / {0, 1, 2, 3, 4, 5, 6}, where M is U(10, 10): Matroid of rank 10 on 10 elements with circuit-closures
{}, x^2 + 4*x + 1), (M / {0, 1, 2, 3, 4, 5}, where M is U(10, 10): Matroid of rank 10 on 10 elements with circuit-closures
{}, x^3 + 11*x^2 + 11*x + 1), (M / {0, 1, 2, 3, 4}, where M is U(10, 10): Matroid of rank 10 on 10 elements with circuit-closures
{}, x^4 + 26*x^3 + 66*x^2 + 26*x + 1), (M / {0, 1, 2, 3}, where M is U(10, 10): Matroid of rank 10 on 10 elements with circuit-closures
{}, x^5 + 57*x^4 + 302*x^3 + 302*x^2 + 57*x + 1), (M / {0, 1, 2}, where M is U(10, 10): Matroid of rank 10 on 10 elements with circuit-closu

In [ ]:
# Define the uniform matroid U_{2,3}
M = matroids.Uniform(2,3)

def kazhdan_lusztig_polynomial(matroid):
    # Base cases
    if matroid.size() == 0:  # No elements
        return 1
    if not matroid.is_connected():  # Not connected
        return 0

    # Recursive step
    element = list(matroid.groundset())[0]  # Pick any element
    deletion = matroid.delete(element)
    contraction = matroid.contract(element)

    # Compute recursively
    return kazhdan_lusztig_polynomial(deletion) + t * kazhdan_lusztig_polynomial(contraction)


# Define the matroid U_{2,3}
U_2_3 = M

# Create a polynomial ring for computations
R.<t> = PolynomialRing(QQ)

# Compute the Kazhdan-Lusztig polynomial
KL_polynomial = kazhdan_lusztig_polynomial(U_2_3)
print(f"The Kazhdan-Lusztig polynomial of U_{2,3} is: {KL_polynomial}")

The Kazhdan-Lusztig polynomial of U_(2, 3) is: t^3 + 2*t^2 + t
